In [0]:
import os
import ast

def extract_function_names(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        tree = ast.parse(f.read(), filename=filepath)
    return [node.name for node in tree.body if isinstance(node, ast.FunctionDef)]

def generate_init_file(directory):
    lines = []
    all_exports = []
    for filename in sorted(os.listdir(directory)):
        fullpath = os.path.join(directory, filename)
        if filename.endswith(".py") and filename != "__init__.py" and os.path.isfile(fullpath):
            modname = filename[:-3]
            func_names = extract_function_names(fullpath)
            if func_names:
                lines.append(f"from .{modname} import " + ", ".join(func_names))
                all_exports.extend(func_names)
    if all_exports:
        lines.append("")
        lines.append("__all__ = [" + ", ".join(f'"{name}"' for name in all_exports) + "]")
    init_path = os.path.join(directory, "__init__.py")
    with open(init_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines) + "\n")

os.chdir("..")
for layer in [ "functions" ]:
    generate_init_file(layer)